In [1]:
# model segformerb4 settings
from mmseg.models import build_segmentor
from mmcv.runner.optimizer import build_optimizer
import torch

def segformer():
    model = dict(
        type='SunSegmentor',
        backbone=dict(
            type='MixVisionTransformer',
            in_channels=3,
            embed_dims=64,
            num_stages=4,
            num_layers=[2, 2, 2, 2],
            num_heads=[1, 2, 5, 8],
            patch_sizes=[7, 3, 3, 3],
            sr_ratios=[8, 4, 2, 1],
            out_indices=(0, 1, 2, 3),
            mlp_ratio=4,
            qkv_bias=True,
            drop_rate=0.0,
            attn_drop_rate=0.0,
            drop_path_rate=0.1,
            pretrained=None),
        decode_head=dict(
            type='RPFNHead',
            in_channels=[64, 128, 320, 512],
            in_index=[0, 1, 2, 3],
            channels=128,
            dropout_ratio=0.1,
            num_classes=1,
            norm_cfg=dict(type='BN', requires_grad=True),
            align_corners=False,
            loss_decode=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0)),
        # model training and testing settings
        train_cfg=dict(),
        test_cfg=dict(mode='whole'))
    model = build_segmentor(model)
    model.init_weights()
    return model
    

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6
model = segformer()
out = model(torch.rand((2, 3, 352, 352)))
# for i in out:
#     print(i.shape)

/mnt/sdd/nguyen.van.quan/Researchs/Polyp/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(
2022-10-16 23:51:43,760 - mmcv - INFO - initialize RPFNHead with init_cfg {'type': 'Normal', 'std': 0.01, 'override': {'name': 'conv_seg'}}
2022-10-16 23:51:43,768 - mmcv - INFO - 
backbone.layers.0.0.projection.weight - torch.Size([64, 3, 7, 7]): 
Initialized by user-defined `init_weights` in MixVisionTransformer  
 
2022-10-16 23:51:43,768 - mmcv - INFO - 
backbone.layers.0.0.projection.bias - torch.Size([64]): 
Initialized by user-defined `init_weights` in MixVisionTransformer  
 
2022-10-16 23:51:43,769 - mmcv - INFO - 
backbone.layers.0.0.norm.weight - torch.Size([64]): 
The value is the same before and after calling `init_weights` of SunSegmentor  
 
2022-10-1

torch.Size([2, 1, 1, 1])
torch.Size([2, 1, 1, 1])
torch.Size([2, 1, 1, 1])
torch.Size([2, 1, 1, 1])


In [ ]:
import cv2
cv2.__version__

In [ ]:
import torch
a = torch.rand((2, 3, 12, 12))
b = torch.rand((2, 6, 12, 12))

c = torch.add(a,b)
c.shape


In [ ]:
print(x1,x2,x3)

In [ ]:
from torch import nn
import torch
cvb = nn.Conv1d(4, 1 , kernel_size=1)
a = torch.rand((4, 2, 128, 88, 88))
b = torch.rand((1, 2, 128, 1, 1))
c = a + b
c[0].shape

In [ ]:
count_parameters(model)

In [ ]:
from mmseg.models.decode_heads.rcfpn import RCFPN
import torch
rfp = RCFPN([64, 128, 320, 512], 128, 5)
test1 = torch.rand((2, 64, 128, 128))
test2 = torch.rand((2, 128, 64, 64))
test3 = torch.rand((2, 320, 32, 32))
test4 = torch.rand((2, 512, 16, 16))

rfp_out = rfp([test1, test2, test3, test4])


In [ ]:

from mmseg.models.decode_heads.lib.bifpn import *

bifpn = BiFPN([64, 128, 320, 512], 128)

test1 = torch.rand((2, 64, 128, 128))
test2 = torch.rand((2, 128, 64, 64))
test3 = torch.rand((2, 320, 32, 32))
test4 = torch.rand((2, 512, 16, 16))

bifpn_out = bifpn([test1, test2, test3, test4])
for i in bifpn_out:
    print(i.shape)


In [ ]:
for i in range(3, 0, -1):    
    print(i)

In [ ]:

import albumentations as A
from albumentations.pytorch import ToTensorV2
from mcode.custom_transform import *
import glob

val_transform = A.Compose([
    DilationAndErosion(),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

In [ ]:
import torchvision.transforms as transforms
from mcode.custom_transform import *
import glob
train_transform = transforms.Compose([
    Random_dilation_erosion(),
    Random_flip(),
    Random_image_enhance(),
    Random_rotate(),
    Random_scale_crop(),
    Tonumpy(),
    Normalize(),
    Totensor(),
    
])

In [ ]:
from torch.utils.data import Dataset, DataLoader

f_images = glob.glob('TrainDataset/image/*')
f_masks = glob.glob('TrainDataset/mask/*')

X_test = glob.glob('TestDataset/*/images/*')
y_test = glob.glob('TestDataset/*/masks/*')

# X_val = glob.glob('/content/TestDataset/ETIS-LaribPolypDB/images/*')
# y_val = glob.glob('/content/TestDataset/ETIS-LaribPolypDB/masks/*')

trainsize = 352

budget_size = 75

seed = 2022

device = 'cuda:0'
class ActiveDataset(Dataset):
    """
    dataloader for polyp segmentation tasks
    """
    def __init__(self, image_paths=[], gt_paths=[], trainsize=352, transform=None):
        self.trainsize = trainsize
        self.images = image_paths
        self.masks = gt_paths
        self.size = len(self.images)
        self.filter_files()
        self.transform = transform
        
    def __getitem__(self, index):
        image = self.rgb_loader(self.images[index])
        mask = self.binary_loader(self.masks[index])
        if self.transform is not None:
            transformed = self.transform(image=image, mask=mask)
            image = transformed["image"]
            mask = transformed["mask"]
            
        sample = dict(image=image, mask=mask.unsqueeze(0), image_path=self.images[index], mask_path=self.masks[index])
        
        return sample

    def filter_files(self):
        assert len(self.images) == len(self.masks)
        images = []
        masks = []
        for img_path, mask_path in zip(self.images, self.masks):
            img = Image.open(img_path)
            mask = Image.open(mask_path)
            if img.size == mask.size:
                images.append(img_path)
                masks.append(mask_path)
        self.images = images
        self.masks = masks
    
    def rgb_loader(self, path):
        with open(path, 'rb') as f:
            img = Image.open(f).resize((trainsize, trainsize), Image.BILINEAR)
            return np.array(img.convert('RGB'))

    def binary_loader(self, path):
        with open(path, 'rb') as f:
            img = Image.open(f).resize((trainsize, trainsize), Image.NEAREST)
            img = np.array(img.convert('L'))
            return img

    def __len__(self):
        return self.size

In [ ]:


train_dataset = ActiveDataset(
    f_images, 
    f_masks, 
    trainsize=trainsize, 
    transform=val_transform
)


print(f"Train size: {len(train_dataset)}")




In [ ]:
sample = train_dataset[10]

In [ ]:
class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
            # The normalize code -> t.sub_(m).div_(s)
        return tensor
    
unorm = UnNormalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))

In [ ]:
# lets look at some samples
from matplotlib import pyplot as plt
sample = train_dataset[10]


plt.subplot(1,2,1)
plt.imshow(unorm(sample["image"]).permute(1, 2, 0)) # for visualization we have to permute back to HWC
plt.subplot(1,2,2)
plt.imshow(sample["mask"].squeeze())  # for visualization we have to remove 3rd dimension of mask
plt.show()

In [12]:
import torch

a = torch.randint(0, 100, (1780,))

a = a[::, None]
print(a.shape)

b = torch.tensor([10, 20])
c = torch.tensor([5, 6])

a[b] = c

print(a[10], a[20])
print(a.shape, b.shape, c.shape)

torch.Size([1780, 1])


RuntimeError: shape mismatch: value tensor of shape [2] cannot be broadcast to indexing result of shape [2, 1]

In [5]:
import torch
from torch import nn
from torch.nn import functional as F
class ScaleBranch(nn.Module):

    def __init__(self, in_channels=256, out_channels=256):
        super(ScaleBranch, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.spatialPooling = nn.AdaptiveAvgPool3d((2, 1, 1))
        self.scalePooling = nn.AdaptiveAvgPool2d(1)
        self.channel_agg = nn.Conv2d(in_channels, 1, kernel_size=1)
        self.trans = nn.Conv2d(self.in_channels, 1, 1)

    def forward(self, x):
        x = self.spatialPooling(x.permute(0, 2, 1, 3, 4)).reshape(
            x.size(0), x.size(2), 2, 1)
        print(x.shape)
        batch, channel, height, width = x.size() # B, C, 2, 1
        channel_context = self.channel_agg(x)
        print(channel_context.shape)
        channel_context = channel_context.view(batch, 1, height * width)
        print(channel_context.shape)
        channel_context = F.softmax(channel_context, dim=-1)
        channel_context = channel_context * height * width
        channel_context = channel_context.view(batch, 1, height, width)
        res = x * channel_context
        print(res.shape)
        context = self.scalePooling(x * channel_context)

        context = self.trans(context)
        return context
    

model = ScaleBranch()
a = torch.rand((10, 2, 256, 128, 128))
model(a).shape

torch.Size([10, 256, 2, 1])
torch.Size([10, 1, 2, 1])
torch.Size([10, 1, 2])
torch.Size([10, 256, 2, 1])


torch.Size([10, 1, 1, 1])